In [2]:
# Cell 0 — Exp03 (Method 3) SegFormer + Semi-Supervised (EMA Teacher)
from pathlib import Path
import re
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)

# ====== Your provided paths ======
X_TEST_DIR  = Path(r"C:\Users\asus\Desktop\ECN\DEEP\DataChallenge\data\X_test_xNbnvIa")
X_TRAIN_DIR = Path(r"C:\Users\asus\Desktop\ECN\DEEP\DataChallenge\data\X_train_uDRk9z9")
X_UNLAB_DIR = Path(r"C:\Users\asus\Desktop\ECN\DEEP\DataChallenge\data\X_unlabeled_mtkxUlo")
Y_TRAIN_CSV = Path(r"C:\Users\asus\Desktop\ECN\DEEP\DataChallenge\data\Y_train_T9NrBYo.csv")
SAMPLE_SUB  = Path(r"C:\Users\asus\Desktop\ECN\DEEP\DataChallenge\data\submission_csv_file_random_example_3qPSCtv.csv")

# ====== Exp03 output ======
OUT_DIR = Path(r"exp_outputs\Exp03_SegFormer_SemiSupervised")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ====== Segmentation constants ======
NUM_CLASSES  = 3
IGNORE_INDEX = 255
H            = 160
W_PAD        = 288   # pad to 288; later crop back to original width (160/272)

# ====== Load method2 supervised best to initialize (CHANGE if your path differs) ======
SUP_BEST = Path(r"exp_outputs\Exp02_SegFormer_Supervised\best_state_dict.pt")
print("SUP_BEST exists:", SUP_BEST.exists(), SUP_BEST)

torch.backends.cudnn.benchmark = True


DEVICE: cuda
SUP_BEST exists: True exp_outputs\Exp02_SegFormer_Supervised\best_state_dict.pt


In [3]:
# Cell 1 — Utilities: name parsing, file listing (de-dup), X loading, padding, Y loading/padding
NAME_RE = re.compile(r"well_(\d+)_section_(\d+)_patch_(\d+)$")

def parse_name(stem: str):
    m = NAME_RE.match(stem)
    if not m:
        raise ValueError(f"Bad patch name: {stem}")
    return int(m.group(1)), int(m.group(2)), int(m.group(3))

def list_npy_files(dir_path: Path):
    # de-duplicate robustly (prevents Windows *.npy/*.NPY double counting)
    files = list(dir_path.rglob("*.npy")) + list(dir_path.rglob("*.NPY"))
    uniq = sorted({Path(p).resolve() for p in files})
    return [Path(p) for p in uniq]

def load_x(path: Path) -> np.ndarray:
    x = np.load(path)
    if x.ndim == 3 and x.shape[0] == 1:
        x = x[0]
    x = np.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)
    mn, mx = float(x.min()), float(x.max())
    if mx > mn:
        x = (x - mn) / (mx - mn)
    else:
        x = np.zeros_like(x, dtype=np.float32)
    return x  # (160,w)

def pad_x_to_wpad(x: np.ndarray) -> np.ndarray:
    h, w = x.shape
    out = np.zeros((h, W_PAD), dtype=np.float32)
    out[:, :w] = x
    return out

y_df = pd.read_csv(Y_TRAIN_CSV, index_col=0)

def restore_mask_from_row(row_values: np.ndarray) -> np.ndarray:
    vals = row_values[row_values != -1]
    return vals.reshape(H, -1).astype(np.int64)  # (160,160) or (160,272)

def pad_mask_to_wpad(mask: np.ndarray, w: int) -> np.ndarray:
    out = np.full((H, W_PAD), IGNORE_INDEX, dtype=np.int64)
    out[:, :w] = mask
    return out

def make_valid_mask(w: int) -> np.ndarray:
    valid = np.zeros((H, W_PAD), dtype=np.bool_)
    valid[:, :w] = True
    return valid


In [4]:
# Cell 2 — Build manifests + split labeled train/val by well
def build_manifest(x_dir: Path) -> pd.DataFrame:
    rows = []
    for p in list_npy_files(x_dir):
        stem = p.stem
        try:
            well, section, patch = parse_name(stem)
        except ValueError:
            continue
        arr = np.load(p, mmap_mode="r")
        if arr.ndim == 3 and arr.shape[0] == 1:
            w = int(arr.shape[2])
        elif arr.ndim == 2:
            w = int(arr.shape[1])
        else:
            raise ValueError(f"Unexpected shape {arr.shape} for {p}")
        rows.append({"name": stem, "well": well, "section": section, "patch": patch, "w": w, "path": str(p)})
    return pd.DataFrame(rows)

train_df = build_manifest(X_TRAIN_DIR)
unlab_df = build_manifest(X_UNLAB_DIR)
test_df  = build_manifest(X_TEST_DIR)

# keep only labeled train patches
train_df = train_df[train_df["name"].isin(y_df.index)].reset_index(drop=True)

print("train labeled:", len(train_df), "w:", train_df["w"].value_counts().to_dict(), "wells:", sorted(train_df["well"].unique()))
print("unlabeled   :", len(unlab_df), "w:", unlab_df["w"].value_counts().to_dict())
print("test        :", len(test_df),  "w:", test_df["w"].value_counts().to_dict())

# split by well (you can change VAL_WELLS)
VAL_WELLS = {5}
train_split = train_df[~train_df["well"].isin(VAL_WELLS)].reset_index(drop=True)
val_split   = train_df[train_df["well"].isin(VAL_WELLS)].reset_index(drop=True)
print("train_split:", len(train_split), "val_split:", len(val_split))


train labeled: 4410 w: {272: 3726, 160: 684} wells: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6)]
unlabeled   : 1980 w: {160: 1476, 272: 504}
test        : 972 w: {272: 972}
train_split: 4122 val_split: 288


In [5]:
# Cell 3 — Datasets & DataLoaders
class LabeledSegDataset(Dataset):
    """Returns: x(1,H,W_PAD), y(H,W_PAD), valid(H,W_PAD), meta"""
    def __init__(self, df: pd.DataFrame, y_df: pd.DataFrame, train_mode: bool, seed: int = 42):
        self.df = df.reset_index(drop=True)
        self.y_df = y_df
        self.train_mode = train_mode
        self.rng = np.random.RandomState(seed)

    def __len__(self): 
        return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        name = row["name"]
        w = int(row["w"])

        x = load_x(Path(row["path"]))      # (160,w)
        x = pad_x_to_wpad(x)               # (160,288)

        y_raw = restore_mask_from_row(self.y_df.loc[name].values)
        y = pad_mask_to_wpad(y_raw, w=w)   # (160,288)

        # safe aug: horizontal flip (geometry) — apply to both x/y
        if self.train_mode and self.rng.rand() < 0.5:
            x = np.flip(x, axis=1).copy()
            y = np.flip(y, axis=1).copy()

        x_t = torch.from_numpy(x).unsqueeze(0)          # (1,160,288)
        y_t = torch.from_numpy(y).long()                # (160,288)
        valid_t = torch.from_numpy(make_valid_mask(w))  # (160,288)

        meta = {"name": name, "well": int(row["well"]), "orig_w": w}
        return x_t, y_t, valid_t, meta

class UnlabeledSegDataset(Dataset):
    """Returns: x(1,H,W_PAD), valid(H,W_PAD), meta"""
    def __init__(self, df: pd.DataFrame):
        self.df = df.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        name = row["name"]
        w = int(row["w"])

        x = load_x(Path(row["path"]))      # (160,w)
        x = pad_x_to_wpad(x)               # (160,288)

        x_t = torch.from_numpy(x).unsqueeze(0)          # (1,160,288)
        valid_t = torch.from_numpy(make_valid_mask(w))  # (160,288)
        meta = {"name": name, "orig_w": w}
        return x_t, valid_t, meta

train_loader = DataLoader(
    LabeledSegDataset(train_split, y_df, train_mode=True, seed=123),
    batch_size=8, shuffle=True, num_workers=0, pin_memory=(DEVICE=="cuda")
)
val_loader = DataLoader(
    LabeledSegDataset(val_split, y_df, train_mode=False, seed=123),
    batch_size=8, shuffle=False, num_workers=0, pin_memory=(DEVICE=="cuda")
)
unlab_loader = DataLoader(
    UnlabeledSegDataset(unlab_df),
    batch_size=8, shuffle=True, num_workers=0, pin_memory=(DEVICE=="cuda")
)

# quick sanity
xb, yb, vb, mb = next(iter(train_loader))
xu, vu, mu = next(iter(unlab_loader))
print("labeled x:", xb.shape, "y:", yb.shape, "valid:", vb.shape)
print("unlab   x:", xu.shape, "valid:", vu.shape)


labeled x: torch.Size([8, 1, 160, 288]) y: torch.Size([8, 160, 288]) valid: torch.Size([8, 160, 288])
unlab   x: torch.Size([8, 1, 160, 288]) valid: torch.Size([8, 160, 288])


In [6]:
# Cell 4 — Loss/Metric + logit upsampling
# labeled loss: CE(weighted) + Dice; unlabeled pseudo loss: CE(unweighted)
ce_weights = torch.tensor([1.0, 3.0, 4.0], dtype=torch.float32).to(DEVICE)
ce_l = nn.CrossEntropyLoss(weight=ce_weights, ignore_index=IGNORE_INDEX)
ce_u = nn.CrossEntropyLoss(ignore_index=IGNORE_INDEX)

def upsample_logits(logits: torch.Tensor, target_hw) -> torch.Tensor:
    return F.interpolate(logits, size=target_hw, mode="bilinear", align_corners=False)

def soft_dice_loss(logits: torch.Tensor, target: torch.Tensor, smooth: float = 1.0) -> torch.Tensor:
    probs = torch.softmax(logits, dim=1)
    valid = (target != IGNORE_INDEX).unsqueeze(1)

    t = target.clone()
    t[t == IGNORE_INDEX] = 0
    onehot = F.one_hot(t, num_classes=NUM_CLASSES).permute(0,3,1,2).float()

    probs = probs * valid
    onehot = onehot * valid

    inter = (probs * onehot).sum((0,2,3))
    denom = (probs + onehot).sum((0,2,3))
    dice = (2*inter + smooth) / (denom + smooth)
    return 1.0 - dice.mean()

def labeled_loss(logits_up: torch.Tensor, y: torch.Tensor, dice_w: float = 0.5) -> torch.Tensor:
    return (1-dice_w) * ce_l(logits_up, y) + dice_w * soft_dice_loss(logits_up, y)

def mean_iou(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    valid = (target != IGNORE_INDEX)
    ious = []
    for c in range(NUM_CLASSES):
        p = (pred == c) & valid
        t = (target == c) & valid
        inter = (p & t).sum().float()
        union = (p | t).sum().float()
        ious.append(torch.tensor(1.0, device=pred.device) if union.item() == 0 else inter / union)
    return torch.stack(ious).mean()


In [7]:
# Cell 5 — Model: student + EMA teacher (SegFormer), init from supervised best
from transformers import SegformerForSemanticSegmentation

BACKBONE = "nvidia/segformer-b2-finetuned-ade-512-512"

student = SegformerForSemanticSegmentation.from_pretrained(
    BACKBONE, num_labels=NUM_CLASSES, ignore_mismatched_sizes=True
).to(DEVICE)

teacher = SegformerForSemanticSegmentation.from_pretrained(
    BACKBONE, num_labels=NUM_CLASSES, ignore_mismatched_sizes=True
).to(DEVICE)

# init both from supervised checkpoint (method2 best)
state = torch.load(SUP_BEST, map_location=DEVICE)
student.load_state_dict(state)
teacher.load_state_dict(state)

teacher.eval()
for p in teacher.parameters():
    p.requires_grad_(False)

print("Initialized student/teacher from:", SUP_BEST)


Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b2-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([150, 768, 1, 1]) in the checkpoint and torch.Size([3, 768, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b2-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([150, 768, 1, 1]) in the checkpoint and torch.Size([3, 768, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found shape torch.Size([150]) in 

Initialized student/teacher from: exp_outputs\Exp02_SegFormer_Supervised\best_state_dict.pt


In [8]:
# Cell 6 — Semi-supervised training (EMA Teacher + high-conf pseudo labels)
def rampup(epoch: int, ramp_epochs: int = 5) -> float:
    if epoch < ramp_epochs:
        return float(epoch + 1) / float(ramp_epochs)
    return 1.0

@torch.no_grad()
def ema_update(teacher_model, student_model, alpha: float):
    # EMA for parameters
    for t_p, s_p in zip(teacher_model.parameters(), student_model.parameters()):
        t_p.data.mul_(alpha).add_(s_p.data, alpha=1.0 - alpha)
    # buffers: copy (stable)
    for t_b, s_b in zip(teacher_model.buffers(), student_model.buffers()):
        t_b.copy_(s_b)

def weak_aug(x: torch.Tensor) -> torch.Tensor:
    # x: (B,1,H,W) in [0,1]
    B = x.size(0)
    a = torch.empty((B,1,1,1), device=x.device).uniform_(0.95, 1.05)
    b = torch.empty((B,1,1,1), device=x.device).uniform_(-0.03, 0.03)
    return torch.clamp(x * a + b, 0.0, 1.0)

def strong_aug(x: torch.Tensor) -> torch.Tensor:
    # intensity + noise + cutout (NO geometry to keep pseudo-label alignment)
    B, _, Hh, Ww = x.shape
    a = torch.empty((B,1,1,1), device=x.device).uniform_(0.85, 1.15)
    b = torch.empty((B,1,1,1), device=x.device).uniform_(-0.08, 0.08)
    out = torch.clamp(x * a + b, 0.0, 1.0)
    sigma = torch.empty((B,1,1,1), device=x.device).uniform_(0.0, 0.06)
    out = torch.clamp(out + torch.randn_like(out) * sigma, 0.0, 1.0)

    # cutout per-sample
    for i in range(B):
        if torch.rand((), device=x.device).item() < 0.5:
            ch = int(torch.randint(low=10, high=50, size=(1,), device=x.device).item())
            cw = int(torch.randint(low=10, high=80, size=(1,), device=x.device).item())
            y0 = int(torch.randint(low=0, high=Hh-ch+1, size=(1,), device=x.device).item())
            x0 = int(torch.randint(low=0, high=Ww-cw+1, size=(1,), device=x.device).item())
            out[i, :, y0:y0+ch, x0:x0+cw] = 0.0
    return out

# Hyperparams (you can tune)
EPOCHS    = 10
LR        = 6e-5
WEIGHT_DECAY = 0.01
TAU       = 0.95      # pseudo-label confidence threshold
LAMBDA_U  = 1.0       # max weight for unlabeled loss
RAMP_E    = 5         # ramp-up epochs for unlabeled weight
EMA_ALPHA = 0.996     # teacher EMA momentum

best_path = OUT_DIR / "best_state_dict.pt"
opt = torch.optim.AdamW(student.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))

best_miou = -1.0

# iterator helper
def cycle_loader(loader):
    while True:
        for batch in loader:
            yield batch

unlab_iter = cycle_loader(unlab_loader)

for ep in range(1, EPOCHS+1):
    student.train()
    teacher.eval()

    lam_u = LAMBDA_U * rampup(ep-1, ramp_epochs=RAMP_E)

    tr_loss, tr_l, tr_u, n = 0.0, 0.0, 0.0, 0
    pbar = tqdm(train_loader, desc=f"[Exp03] train ep{ep} (lam_u={lam_u:.2f})", leave=False)

    for x_l, y_l, valid_l, meta_l in pbar:
        x_u, valid_u, meta_u = next(unlab_iter)

        x_l = x_l.to(DEVICE)                 # (B,1,160,288)
        y_l = y_l.to(DEVICE)                 # (B,160,288)
        valid_u = valid_u.to(DEVICE)         # (B,160,288) bool
        x_u = x_u.to(DEVICE)                 # (B,1,160,288)

        # 3-ch for SegFormer
        x_l3 = x_l.repeat(1,3,1,1)           # (B,3,160,288)

        # unlabeled: weak for teacher, strong for student
        x_u_w = weak_aug(x_u).repeat(1,3,1,1)
        x_u_s = strong_aug(x_u).repeat(1,3,1,1)

        opt.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
            # ----- labeled loss -----
            logits_l = student(pixel_values=x_l3).logits
            logits_l = upsample_logits(logits_l, y_l.shape[-2:])      # (B,C,160,288)
            loss_l = labeled_loss(logits_l, y_l, dice_w=0.5)

            # ----- pseudo labels from teacher (weak) -----
            with torch.no_grad():
                logits_t = teacher(pixel_values=x_u_w).logits
                logits_t = upsample_logits(logits_t, (H, W_PAD))
                probs_t = torch.softmax(logits_t, dim=1)
                conf, pseudo = torch.max(probs_t, dim=1)             # (B,160,288)

                # mask: high confidence & valid (exclude padded columns)
                mask = (conf >= TAU) & valid_u
                pseudo_pl = pseudo.clone()
                pseudo_pl[~mask] = IGNORE_INDEX

            # ----- unlabeled loss on student (strong) -----
            if lam_u > 0:
                logits_u = student(pixel_values=x_u_s).logits
                logits_u = upsample_logits(logits_u, (H, W_PAD))
                loss_u = ce_u(logits_u, pseudo_pl)
            else:
                loss_u = torch.tensor(0.0, device=DEVICE)

            loss = loss_l + lam_u * loss_u

        scaler.scale(loss).backward()
        scaler.step(opt)
        scaler.update()

        # EMA teacher update (after student step)
        ema_update(teacher, student, alpha=EMA_ALPHA)

        bs = x_l.size(0)
        tr_loss += float(loss.item()) * bs
        tr_l    += float(loss_l.item()) * bs
        tr_u    += float(loss_u.item()) * bs
        n += bs
        pbar.set_postfix({"loss": tr_loss/max(1,n), "L": tr_l/max(1,n), "U": tr_u/max(1,n)})

    # ----- validation on labeled val -----
    student.eval()
    miou_sum, n = 0.0, 0
    with torch.no_grad():
        for x, y, valid, meta in tqdm(val_loader, desc=f"[Exp03] val ep{ep}", leave=False):
            x = x.to(DEVICE)                 # (B,1,160,288)
            y = y.to(DEVICE)                 # (B,160,288)
            x3 = x.repeat(1,3,1,1)
            logits = student(pixel_values=x3).logits
            logits = upsample_logits(logits, y.shape[-2:])
            pred = torch.argmax(logits, dim=1)
            miou_sum += float(mean_iou(pred, y).item()) * x.size(0)
            n += x.size(0)
    val_miou = miou_sum / max(1, n)
    print(f"[Exp03] ep{ep:02d}/{EPOCHS}  val_mIoU={val_miou:.4f}")

    if val_miou > best_miou:
        best_miou = val_miou
        torch.save(student.state_dict(), best_path)

print("[Exp03] BEST val mIoU:", best_miou, "saved:", best_path)


C:\Users\asus\AppData\Local\Temp\ipykernel_22368\3700262783.py:53: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))
[Exp03] train ep1 (lam_u=0.20):   0%|          | 0/516 [00:00<?, ?it/s]C:\Users\asus\AppData\Local\Temp\ipykernel_22368\3700262783.py:90: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):


[Exp03] ep01/10  val_mIoU=0.7957


[Exp03] ep02/10  val_mIoU=0.8044


[Exp03] ep03/10  val_mIoU=0.8079


[Exp03] ep04/10  val_mIoU=0.8010


...

[Exp03] ep06/10  val_mIoU=0.7977


[Exp03] ep07/10  val_mIoU=0.8112


[Exp03] ep08/10  val_mIoU=0.7996


[Exp03] ep09/10  val_mIoU=0.8012


[Exp03] ep10/10  val_mIoU=0.7968
[Exp03] BEST val mIoU: 0.8112312638097339 saved: exp_outputs\Exp03_SegFormer_SemiSupervised\best_state_dict.pt


In [9]:
# Cell 7 — Predict X_test strictly by SAMPLE_SUB order + save npy + build correct submission CSV
# 1) ordered names from sample
sample = pd.read_csv(SAMPLE_SUB)
name_col = sample.columns[0]
ordered_names_raw = sample[name_col].astype(str).tolist()

def norm_name(s: str) -> str:
    s = str(s).strip()
    if s.lower().endswith(".npy"):
        s = s[:-4]
    return s

ordered_names = [norm_name(n) for n in ordered_names_raw]
print("sample rows:", len(ordered_names), "name_col:", name_col)

# 2) build test index (de-dup)
test_files = list_npy_files(X_TEST_DIR)
test_index = {p.stem: p for p in test_files}
test_index.update({p.stem.lower(): p for p in test_files})

# 3) load best student
student.load_state_dict(torch.load(best_path, map_location=DEVICE))
student.eval()
print("Loaded best:", best_path)

pred_dir = OUT_DIR / "test_predictions"
pred_dir.mkdir(parents=True, exist_ok=True)
for p in pred_dir.glob("*.npy"):
    p.unlink()

with torch.no_grad():
    for name in tqdm(ordered_names, desc="[Exp03] predict test", leave=False):
        key = name if name in test_index else name.lower()
        if key not in test_index:
            hits = list(X_TEST_DIR.rglob(f"{name}.npy")) + list(X_TEST_DIR.rglob(f"{name}.NPY"))
            if len(hits) == 0:
                raise FileNotFoundError(f"X_test missing: {name}.npy")
            x_path = hits[0]
        else:
            x_path = test_index[key]

        x = load_x(x_path)           # (160,w)
        w = x.shape[1]
        x_pad = pad_x_to_wpad(x)     # (160,288)
        x_t = torch.from_numpy(x_pad).unsqueeze(0).unsqueeze(0).to(DEVICE)  # (1,1,160,288)
        x_t = x_t.repeat(1,3,1,1)    # (1,3,160,288)

        logits = student(pixel_values=x_t).logits
        logits = upsample_logits(logits, (H, W_PAD))
        pred = torch.argmax(logits, dim=1).squeeze(0).cpu().numpy().astype(np.int64)  # (160,288)
        pred = pred[:, :w]  # crop back to original width
        np.save(pred_dir / f"{name}.npy", pred)

print("saved npy predictions to:", pred_dir)

# 4) build submission exactly matching sample format
size_labels = 272
flat_len = H * size_labels

pred_map = {}
for p in pred_dir.glob("*.npy"):
    nm = p.stem
    pred = np.load(p)
    if pred.shape[1] != size_labels:
        aux = -1 + np.zeros(flat_len, dtype=np.int64)
        aux[0:H*H] = pred.flatten()
    else:
        aux = pred.flatten().astype(np.int64)
    pred_map[nm] = aux

missing = [n for n in ordered_names if n not in pred_map]
assert len(missing) == 0, f"missing predictions: {missing[:10]}"

data = np.stack([pred_map[n] for n in ordered_names], axis=0)  # (972, 43520)
col_names = [str(i) for i in range(flat_len)]
sub_df = pd.DataFrame(data, columns=col_names)
sub_df.insert(0, name_col, ordered_names_raw)

out_csv = OUT_DIR / "y_test_submission_MATCH_SAMPLE.csv"
sub_df.to_csv(out_csv, index=False)
print("Saved submission:", out_csv, "shape:", sub_df.shape)


sample rows: 972 name_col: Unnamed: 0


C:\Users\asus\AppData\Local\Temp\ipykernel_22368\1968903237.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(best_path, map_location=D

Loaded best: exp_outputs\Exp03_SegFormer_SemiSupervised\best_state_dict.pt


saved npy predictions to: exp_outputs\Exp03_SegFormer_SemiSupervised\test_predictions
Saved submission: exp_outputs\Exp03_SegFormer_SemiSupervised\y_test_submission_MATCH_SAMPLE.csv shape: (972, 43521)
